In [ ]:
# ============================================================
# CELL 1: Setup
# ============================================================
import json, re, torch, gc, os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# ── UPDATE THESE PATHS ──────────────────────────────────────
# Input: output from 03_batch_inference.ipynb
PATIENT_FILE = '/kaggle/input/loopguard-ai-outputs/patients_with_ai_final.json'
ADAPTER_DIR  = '/kaggle/input/medgemma-loopguard-v2/transformers/default/1'
BASE_MODEL   = 'google/medgemma-1.5-4b-it'
OUTPUT_FILE  = '/kaggle/working/patients_with_ai_final.json'  # overwrite with enriched version
# ────────────────────────────────────────────────────────────

BNB_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
)

print('✅ Config ready')
print(f'   VRAM: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated())/1e9:.2f} GB free')

In [ ]:
# ============================================================
# CELL 2: Load patients_with_ai_final.json
# This is the output from 03_batch_inference — already has ai_analysis
# ============================================================
with open(PATIENT_FILE) as f:
    data = json.load(f)

# Handle both top-level formats
if 'patient_scenarios' in data:
    patients = data['patient_scenarios']
elif isinstance(data, list):
    patients = data
else:
    patients = list(data.values())

print(f'✅ Loaded {len(patients)} patients')
print(f'   Keys: {list(patients[0].keys())}')
print(f'   ai_analysis present: {"ai_analysis" in patients[0]}')
if 'ai_analysis' in patients[0]:
    print(f'   ai_analysis keys: {list(patients[0]["ai_analysis"].keys())}')

In [ ]:
# ============================================================
# CELL 3: Agent 1 — Rule-based Pre-validator (no GPU needed)
# Runs BEFORE model loading — fast, pure Python
# ============================================================

MEDICAL_KEYWORDS = [
    'patient', 'presents', 'diagnosis', 'assessment', 'plan',
    'history', 'symptoms', 'ordered', 'pmh', 'vitals', 'exam',
    'subjective', 'objective', 'labs', 'impression'
]

REQUIRED_FIELDS = [
    'primary_hypothesis', 'differential_diagnoses', 'key_supporting_evidence',
    'urgency_level', 'tests_ordered', 'clinical_reasoning'
]

def agent1_validate(patient: dict) -> dict:
    """
    Rule-based pre-validator.
    Checks: note quality, required field presence, urgency validity.
    """
    issues = []
    note_text = patient['clinical_note']['text']
    ai = patient.get('ai_analysis', {})

    # Check 1: Note length
    if len(note_text) < 100:
        issues.append('Note too short for reliable extraction (<100 chars)')

    # Check 2: Clinical content present
    note_lower = note_text.lower()
    if not any(kw in note_lower for kw in MEDICAL_KEYWORDS):
        issues.append('No clinical content detected in note')

    # Check 3: All 6 fields populated in ai_analysis
    missing = [f for f in REQUIRED_FIELDS if not ai.get(f)]
    if missing:
        issues.append(f'Missing fields in extraction: {missing}')

    # Check 4: Urgency is valid
    urgency = ai.get('urgency_level', '')
    if urgency not in ('high', 'medium', 'low'):
        issues.append(f'Invalid urgency value: "{urgency}"')

    # Check 5: Primary hypothesis not a placeholder
    hypothesis = ai.get('primary_hypothesis', '')
    if len(hypothesis) < 5 or hypothesis.lower() in ('unknown', 'none', 'n/a', 'main diagnosis'):
        issues.append('Primary hypothesis appears to be placeholder or empty')

    return {
        'passed': len(issues) == 0,
        'issues': issues
    }


# Run Agent 1 on all patients
print('Running Agent 1 (rule-based validator)...\n')
a1_results = {}
all_passed = 0

for p in patients:
    pid = p['patient_id']
    result = agent1_validate(p)
    a1_results[pid] = result
    tick = '✅' if result['passed'] else '⚠️'
    issues_str = '' if result['passed'] else f" → {result['issues']}"
    print(f'  {tick} {pid}{issues_str}')
    if result['passed']:
        all_passed += 1

print(f'\nAgent 1: {all_passed}/20 passed validation')

In [ ]:
# ============================================================
# CELL 4: Load Models
# Base MedGemma → Agents 3 & 4
# Fine-tuned v2  → Agent 2 (re-extraction if Agent 1 failed)
# ============================================================
print('Loading base MedGemma (for Agents 3 & 4)...')
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=BNB_CONFIG,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
base_model.eval()
print(f'✅ Base MedGemma loaded | VRAM: {torch.cuda.memory_allocated()/1e9:.2f} GB')

print('\nLoading LoopGuard v2 adapter (for Agent 2 re-extraction)...')
ft_model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
ft_model.eval()
print(f'✅ LoopGuard v2 ready | VRAM: {torch.cuda.memory_allocated()/1e9:.2f} GB')

In [ ]:
# ============================================================
# CELL 5: Agent 2 — Re-extraction for failed Agent 1 cases
# Uses fine-tuned LoopGuard v2
# If Agent 1 passed → skip (use existing ai_analysis)
# If Agent 1 failed → re-run inference
# ============================================================

FIELD_PATTERNS = [
    ('primary_hypothesis',      r'PRIMARY HYPOTHESIS:'),
    ('differential_diagnoses',  r'DIFFERENTIAL DIAGNOSES:'),
    ('key_supporting_evidence', r'KEY SUPPORTING EVIDENCE:'),
    ('urgency_level',           r'URGENCY LEVEL:'),
    ('tests_ordered',           r'TESTS ORDERED:'),
    ('clinical_reasoning',      r'CLINICAL REASONING:'),
]
FIELD_LABEL_RE = (
    r'(?:PRIMARY HYPOTHESIS|DIFFERENTIAL DIAGNOSES|KEY SUPPORTING EVIDENCE|'
    r'URGENCY LEVEL|TESTS ORDERED|CLINICAL REASONING):'
)

def decode_output(raw: str) -> str:
    if 'model\n' in raw:
        return raw.split('model\n')[-1].strip()
    return raw.strip()

def parse_fields(text: str) -> dict:
    clean = re.sub(r'\*\*', '', text)
    result = {}
    for field_key, pattern in FIELD_PATTERNS:
        m = re.search(pattern, clean, re.IGNORECASE)
        if m:
            start = m.end()
            next_m = re.search(FIELD_LABEL_RE, clean[start:], re.IGNORECASE)
            end = start + next_m.start() if next_m else start + 800
            result[field_key] = clean[start:end].strip().strip('[]')
        else:
            result[field_key] = ''
    u = result.get('urgency_level', '').lower()
    result['urgency_level'] = 'high' if 'high' in u else ('low' if 'low' in u else 'medium')
    return result

def agent2_extract(note_text: str, model) -> dict:
    prompt = (
        '<start_of_turn>user\n'
        'Extract diagnostic information from this clinical note.\n\n'
        f'Clinical Note:\n{note_text}\n\n'
        'Output ONLY these 6 fields:\n'
        'PRIMARY HYPOTHESIS: [main diagnosis]\n'
        'DIFFERENTIAL DIAGNOSES: [comma-separated alternatives]\n'
        'KEY SUPPORTING EVIDENCE: [comma-separated findings]\n'
        'URGENCY LEVEL: [high/medium/low]\n'
        'TESTS ORDERED: [comma-separated tests]\n'
        'CLINICAL REASONING: [brief explanation]'
        '<end_of_turn>\n<start_of_turn>model\n'
    )
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1024).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs, max_new_tokens=600, do_sample=False,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
    raw = tokenizer.decode(out[0], skip_special_tokens=True)
    return parse_fields(decode_output(raw))


print('Agent 2 — re-extraction for Agent 1 failures...\n')
a2_rerun_count = 0

for p in patients:
    pid = p['patient_id']
    if not a1_results[pid]['passed']:
        print(f'  Re-extracting {pid}...', end=' ')
        note_text = p['clinical_note']['text']
        new_extraction = agent2_extract(note_text, ft_model)
        p['ai_analysis'].update(new_extraction)
        a2_rerun_count += 1
        fields_ok = sum(1 for v in new_extraction.values() if v)
        print(f'✅ {fields_ok}/6 fields')

if a2_rerun_count == 0:
    print('  All 20 passed Agent 1 — no re-extraction needed ✅')
else:
    print(f'\nAgent 2: re-extracted {a2_rerun_count} patients')

In [ ]:
# ============================================================
# CELL 6: Agent 3 — Quality Checker (base MedGemma)
# Reviews each extraction: urgency match, logical reasoning,
# plausible differentials. Outputs PASS or FAIL: [reason]
# ============================================================

AGENT3_PROMPT = """You are a medical quality reviewer checking an AI-generated diagnostic extraction.

Review this extraction against the original clinical note and check:
1. Does the urgency level match the clinical severity?
2. Does the reasoning logically follow from the key evidence?
3. Are the differential diagnoses clinically plausible?
4. Is the primary hypothesis consistent with the presented symptoms?

Clinical Note:
{note}

AI Extraction:
PRIMARY HYPOTHESIS: {primary_hypothesis}
DIFFERENTIAL DIAGNOSES: {differential_diagnoses}
KEY SUPPORTING EVIDENCE: {key_supporting_evidence}
URGENCY LEVEL: {urgency_level}
TESTS ORDERED: {tests_ordered}
CLINICAL REASONING: {clinical_reasoning}

Output exactly one of:
PASS
FAIL: [specific issue found]"""


def agent3_review(note_text: str, ai: dict, model) -> dict:
    prompt_text = AGENT3_PROMPT.format(
        note=note_text[:800],  # truncate long notes
        primary_hypothesis=ai.get('primary_hypothesis', ''),
        differential_diagnoses=ai.get('differential_diagnoses', ''),
        key_supporting_evidence=ai.get('key_supporting_evidence', ''),
        urgency_level=ai.get('urgency_level', ''),
        tests_ordered=ai.get('tests_ordered', ''),
        clinical_reasoning=ai.get('clinical_reasoning', ''),
    )
    prompt = (
        f'<start_of_turn>user\n{prompt_text}<end_of_turn>\n<start_of_turn>model\n'
    )
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1536).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs, max_new_tokens=80, do_sample=False,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
    raw = tokenizer.decode(out[0], skip_special_tokens=True)
    generated = decode_output(raw)

    # Parse PASS / FAIL
    gen_upper = generated.strip().upper()
    if gen_upper.startswith('PASS'):
        return {'passed': True, 'issue': None, 'raw': generated}
    elif 'FAIL' in gen_upper:
        # Extract reason after 'FAIL:'
        m = re.search(r'FAIL[:\s]+(.+)', generated, re.IGNORECASE | re.DOTALL)
        issue = m.group(1).strip()[:200] if m else 'Quality check failed'
        return {'passed': False, 'issue': issue, 'raw': generated}
    else:
        # Ambiguous output — treat as pass but note it
        return {'passed': True, 'issue': None, 'raw': generated}


print('Running Agent 3 (quality checker) on all 20 patients...\n')
a3_results = {}

for i, p in enumerate(patients):
    pid = p['patient_id']
    note_text = p['clinical_note']['text']
    ai = p['ai_analysis']

    print(f'  [{i+1:02d}/20] {pid}...', end=' ', flush=True)
    result = agent3_review(note_text, ai, base_model)
    a3_results[pid] = result

    tick = '✅ PASS' if result['passed'] else f'⚠️  FAIL: {result["issue"][:60]}'
    print(tick)

passed_count = sum(1 for r in a3_results.values() if r['passed'])
print(f'\nAgent 3: {passed_count}/20 passed quality review')

In [ ]:
# ============================================================
# CELL 7: Agent 4 — Confidence Scorer (base MedGemma)
# Rates diagnostic confidence 1-10
# Flags atypical presentations, conflicting findings, rare diagnoses
# ============================================================

AGENT4_PROMPT = """You are assessing diagnostic confidence for an AI-generated clinical extraction.

Rate the confidence that the primary hypothesis is correct, and flag if review is needed.
Flag if: atypical presentation, conflicting findings, rare diagnosis, or critical urgency mismatch.

Clinical Note (summary):
{note}

AI Extraction:
PRIMARY HYPOTHESIS: {primary_hypothesis}
URGENCY LEVEL: {urgency_level}
KEY SUPPORTING EVIDENCE: {key_supporting_evidence}
CLINICAL REASONING: {clinical_reasoning}

Output exactly:
CONFIDENCE: [1-10]
FLAG: [yes/no]
REASON: [one sentence if flagged, none if not]"""


def agent4_score(note_text: str, ai: dict, model) -> dict:
    prompt_text = AGENT4_PROMPT.format(
        note=note_text[:600],
        primary_hypothesis=ai.get('primary_hypothesis', ''),
        urgency_level=ai.get('urgency_level', ''),
        key_supporting_evidence=ai.get('key_supporting_evidence', '')[:200],
        clinical_reasoning=ai.get('clinical_reasoning', '')[:200],
    )
    prompt = f'<start_of_turn>user\n{prompt_text}<end_of_turn>\n<start_of_turn>model\n'
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1536).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs, max_new_tokens=60, do_sample=False,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
    raw = tokenizer.decode(out[0], skip_special_tokens=True)
    generated = decode_output(raw)

    # Parse CONFIDENCE, FLAG, REASON
    confidence = 7  # default
    flagged = False
    reason = 'none'

    m_conf = re.search(r'CONFIDENCE[:\s]+([0-9]+(?:\.[0-9]+)?)', generated, re.IGNORECASE)
    if m_conf:
        try:
            confidence = min(10, max(1, int(float(m_conf.group(1)))))
        except:
            confidence = 7

    m_flag = re.search(r'FLAG[:\s]+(yes|no)', generated, re.IGNORECASE)
    if m_flag:
        flagged = m_flag.group(1).lower() == 'yes'

    m_reason = re.search(r'REASON[:\s]+(.+)', generated, re.IGNORECASE | re.DOTALL)
    if m_reason:
        reason_text = m_reason.group(1).strip()[:200]
        reason = reason_text if reason_text.lower() != 'none' else 'none'

    return {
        'confidence': confidence,
        'flagged': flagged,
        'reason': reason,
        'raw': generated
    }


print('Running Agent 4 (confidence scorer) on all 20 patients...\n')
a4_results = {}

for i, p in enumerate(patients):
    pid = p['patient_id']
    note_text = p['clinical_note']['text']
    ai = p['ai_analysis']

    print(f'  [{i+1:02d}/20] {pid}...', end=' ', flush=True)
    result = agent4_score(note_text, ai, base_model)
    a4_results[pid] = result

    flag_str = f' 🚩 {result["reason"][:50]}' if result['flagged'] else ''
    print(f'confidence={result["confidence"]}/10 flag={result["flagged"]}{flag_str}')

flagged_count = sum(1 for r in a4_results.values() if r['flagged'])
avg_conf = sum(r['confidence'] for r in a4_results.values()) / len(a4_results)
print(f'\nAgent 4: avg confidence={avg_conf:.1f}/10 | flagged={flagged_count}/20')

In [ ]:
# ============================================================
# CELL 8: Merge Agent Results + Save Final JSON
# Each patient now has:
#   ai_analysis.agent_validation  → Agent 1 result
#   ai_analysis.agent_quality     → Agent 3 result
#   ai_analysis.agent_confidence  → Agent 4 score
#   ai_analysis.agent_review_flag → True if any agent raised concern
# ============================================================

for p in patients:
    pid = p['patient_id']
    ai = p['ai_analysis']

    a1 = a1_results[pid]
    a3 = a3_results[pid]
    a4 = a4_results[pid]

    # Master flag: any agent raised a concern
    review_flag = (
        not a1['passed'] or
        not a3['passed'] or
        a4['flagged'] or
        a4['confidence'] <= 5
    )

    ai['agent_validation'] = {
        'passed': a1['passed'],
        'issues': a1['issues'],
    }
    ai['agent_quality'] = {
        'passed': a3['passed'],
        'issue': a3['issue'],
    }
    ai['agent_confidence'] = a4['confidence']
    ai['agent_flagged'] = a4['flagged']
    ai['agent_flag_reason'] = a4['reason']
    ai['agent_review_flag'] = review_flag

    # Clean up internal debug fields before saving
    ai.pop('_raw_output', None)
    ai.pop('_fields_present', None)


# Save
output = {'patient_scenarios': patients}
with open(OUTPUT_FILE, 'w') as f:
    json.dump(output, f, indent=2)

size_kb = len(json.dumps(output)) / 1024
print(f'✅ Saved → {OUTPUT_FILE} ({size_kb:.1f} KB)')
print(f'   {len(patients)} patients with full agent metadata')

In [ ]:
# ============================================================
# CELL 9: Pipeline Summary Report
# ============================================================
print('=== LOOPGUARD AGENTIC PIPELINE REPORT ===')
print(f'{"Patient":<8} {"A1":<6} {"A3":<6} {"Conf":<6} {"Flag":<6} {"Urgency"}')
print('-' * 55)

for p in patients:
    pid = p['patient_id']
    ai = p['ai_analysis']
    a1_ok = '✅' if ai['agent_validation']['passed'] else '⚠️'
    a3_ok = '✅' if ai['agent_quality']['passed'] else '⚠️'
    conf = ai['agent_confidence']
    flag = '🚩' if ai['agent_review_flag'] else '  '
    urg = ai['urgency_level']
    print(f'{pid:<8} {a1_ok:<6} {a3_ok:<6} {conf:<6} {flag:<6} {urg}')

print()
total_flagged = sum(1 for p in patients if p['ai_analysis']['agent_review_flag'])
avg_conf = sum(p['ai_analysis']['agent_confidence'] for p in patients) / len(patients)
a1_pass = sum(1 for p in patients if p['ai_analysis']['agent_validation']['passed'])
a3_pass = sum(1 for p in patients if p['ai_analysis']['agent_quality']['passed'])

print(f'Agent 1 (validation):  {a1_pass}/20 passed')
print(f'Agent 3 (quality):     {a3_pass}/20 passed')
print(f'Agent 4 (confidence):  avg {avg_conf:.1f}/10')
print(f'Flagged for review:    {total_flagged}/20')
print(f'\n✅ Pipeline complete. Download patients_with_ai_final.json')